In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
!pip install -U albumentations --user

In [30]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [31]:
% cd /content/drive/MyDrive/Colab Notebooks/SPACENET7/Multi-temporal.building.tracker/codes

/content/drive/MyDrive/Colab Notebooks/SPACENET7/Multi-temporal.building.tracker/codes


In [32]:
!pip install rasterio
!pip install geopandas
# Imports
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
from torchvision import datasets, transforms, models
from pathlib import Path
import pathlib
from PIL import Image
import pandas as pd

from data_loader import Dataloader_trdp

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [33]:
% cd /content/drive/MyDrive/Colab Notebooks/SPACENET7/DATASET/dataset_pruebas

/content/drive/MyDrive/Colab Notebooks/SPACENET7/DATASET/dataset_pruebas


In [34]:
# Hyperparameters
in_channel = 3
num_classes = 1
learning_rate = 1e-3
batch_size = 16
num_epochs = 2
chip_dimension = 256

In [35]:
train_dir = Path('../../DATASET/dataset_pruebas/train')
test_dir = Path('../../DATASET/dataset_pruebas/validation')
sample_dir = Path('../../DATASET/SN7_buildings_train_sample')

root_dir  = train_dir
csv_file = Path('../output_csvs/df_train_untidy.csv')
csv_file_test = Path('../output_csvs/df_test_untidy.csv')
df = pd.read_csv(csv_file)
df_test = pd.read_csv(csv_file_test)

In [36]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
A.Normalize(mean=mean,std=std)
A.Rotate(limit=(-360, 360), interpolation=4, border_mode=4,p=1),

chip_dimension = 256

transform = A.Compose(
    [
        A.PadIfNeeded(min_height=chip_dimension,min_width=chip_dimension,value=0,p=1),
        A.RandomRotate90(p=1.0),
        ToTensorV2()
    ]
)

In [37]:
train_set   = Dataloader_trdp(root_dir=root_dir,csv_file=csv_file,chip_dimension=chip_dimension,transform=transform)

testing_set = Dataloader_trdp(root_dir=root_dir,csv_file=csv_file_test,chip_dimension=chip_dimension,transform=transform)

In [38]:
print(f"len(train_set): {len(train_set)}")
#print(f"len(testing_set): {len(testing_set)}")

for i in range(len(train_set)):
    sample = train_set[i]
    # verify dimensionality
    print(sample['raster_diff'].shape, sample['mask_diff'].shape)
    if i==15:
        break

len(train_set): 1504
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])


In [39]:
train_set, test_set = torch.utils.data.random_split(train_set, [1000,504])

In [50]:
train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle = True)


# UNET


In [41]:
% cd /content/drive/MyDrive/Colab Notebooks/SPACENET7/Multi-temporal.building.tracker/codes

/content/drive/MyDrive/Colab Notebooks/SPACENET7/Multi-temporal.building.tracker/codes


In [42]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from model import UNET
from utils import (
    load_checkpoint,
    save_checkpoint,
    #get_loaders,
    check_accuracy,
    save_predictions_as_imgs,
)


In [43]:

# Hyperparameters etc.
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 8
NUM_EPOCHS = 3
NUM_WORKERS = 2
IMAGE_HEIGHT = 160  # 1280 originally
IMAGE_WIDTH = 240  # 1918 originally
PIN_MEMORY = True
LOAD_MODEL = False
'''
TRAIN_IMG_DIR = "data/train_images/"
TRAIN_MASK_DIR = "data/train_masks/"
VAL_IMG_DIR = "data/val_images/"
VAL_MASK_DIR = "data/val_masks/"
'''

'\nTRAIN_IMG_DIR = "data/train_images/"\nTRAIN_MASK_DIR = "data/train_masks/"\nVAL_IMG_DIR = "data/val_images/"\nVAL_MASK_DIR = "data/val_masks/"\n'

In [44]:
def train_fn(loader, model, optimizer, loss_fn, scaler):
    
    loop = tqdm(loader)
    
    for batch_idx, (data) in enumerate(loop):
        
        images = data["raster_diff"].float()
        targets = data["mask_diff"].float()#type(torch.LongTensor)
       # images, mask = images.to(device), mask.to(device).unsqueeze(1)
        
        
        
        images = images.to(device=DEVICE)
        #targets = targets.float().unsqueeze(1).to(device=DEVICE) #
        targets = targets.float().squeeze().to(device=DEVICE) #

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(images).squeeze()
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop: 
        loop.set_postfix(loss=loss.item())
        

In [45]:
train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )


In [46]:
model = UNET(in_channels=3, out_channels=1).to(DEVICE)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [47]:
torch.backends.cudnn.enabled = False

In [48]:
'''
num_correct = 0
num_pixels = 0
dice_score = 0
model.eval()
with torch.no_grad():
        for data in test_loader:
            x = data["raster_diff"].float()
            y = data["mask_diff"].float()
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            print(f"x.shape : {x.shape}, y.shape : {y.shape}")
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                (preds + y).sum() + 1e-8
            )

print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )
print(f"Dice score: {dice_score/len(loader)}")
'''

'\nnum_correct = 0\nnum_pixels = 0\ndice_score = 0\nmodel.eval()\nwith torch.no_grad():\n        for data in test_loader:\n            x = data["raster_diff"].float()\n            y = data["mask_diff"].float()\n            x = x.to(device)\n            y = y.to(device).unsqueeze(1)\n            print(f"x.shape : {x.shape}, y.shape : {y.shape}")\n            preds = torch.sigmoid(model(x))\n            preds = (preds > 0.5).float()\n            num_correct += (preds == y).sum()\n            num_pixels += torch.numel(preds)\n            dice_score += (2 * (preds * y).sum()) / (\n                (preds + y).sum() + 1e-8\n            )\n\nprint(\n        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"\n    )\nprint(f"Dice score: {dice_score/len(loader)}")\n'

In [ ]:
'''
train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )
'''
if LOAD_MODEL:
        load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)
def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()
    with torch.no_grad():
            for data in loader:
                x = data["raster_diff"].float()
                y = data["mask_diff"].float()
                x = x.to(device)
                y = y.to(device).unsqueeze(1)
                print(f"x.shape : {x.shape}, y.shape : {y.shape}")
                preds = torch.sigmoid(model(x))
                preds = (preds > 0.5).float()
                num_correct += (preds == y).sum()
                num_pixels += torch.numel(preds)
                dice_score += (2 * (preds * y).sum()) / (
                    (preds + y).sum() + 1e-8
                )

    print(
            f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
        )
    print(f"Dice score: {dice_score/len(loader)}")
    model.train()
#check_accuracy(test_loader, model, device=DEVICE)
scaler = torch.cuda.amp.GradScaler()


In [51]:
test_loader

In [ ]:

for epoch in range(NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn, scaler)

        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer":optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

        # check accuracy
        check_accuracy(val_loader, model, device=DEVICE)

        # print some examples to a folder
        save_predictions_as_imgs(
            test_loader, model, folder="saved_images/", device=DEVICE
        )




In [ ]:
def save_predictions_as_imgs():#()
#        loader, model, folder="/home/sebasmos/Desktop/TRPD/DATASET/segmented_imgs/", device="cuda"
#):
    model.eval()
    for idx, data in enumerate(loader):
                    
        x = data["raster_diff"].float()
        y = data["mask_diff"].float()
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        torchvision.utils.save_image(preds, f"{folder}/pred_{idx}.png")
        torchvision.utils.save_image(y.unsqueeze(1), f"{folder}{idx}.png")

    model.train()